# Energy Market Forecasting and Trading Decision Model

This notebook presents a **data-driven decision support system** for intraday energy trading, developed in accordance with the [TASK.pdf](./TASK.pdf).  
The project focuses on predicting the **expected settlement price** in the Nordic balancing market and using this forecast to guide trading actions (BUY / SELL / HOLD).  
The evaluation then estimates the **expected profit** from these decisions.

---

## Objective

The goal is to forecast the **expected settlement price** for the delivery hour (17:00–18:00), based on market information available at **16:53**.  
From this expectation, we determine whether the trader should **buy**, **sell**, or **hold** a position.

The model first estimates the **probability** that the system regulation will be *positive* (the market being short).  
Given this probability $ p_{\text{pos}} $, the expected settlement price is calculated as:

$$
\mathbb{E}[P_{\text{settlement}}] = p_{\text{pos}} \times 90 + (1 - p_{\text{pos}}) \times 0,
$$

where:
- €90  represents the settlement price if the system is short (positive regulation),
- €0  represents the price if the system is long (negative regulation).

---

## Decision Logic

Once the expected settlement price is estimated, the trading action is determined by comparing it to the **market bid/offer price** (≈ 30 €):

$$
\text{Decision} =
\begin{cases}
\text{BUY}, & \text{if } \mathbb{E}[P_{\text{settlement}}] > 30,\\
\text{SELL}, & \text{if } \mathbb{E}[P_{\text{settlement}}] < 30,\\
\text{HOLD}, & \text{if } \mathbb{E}[P_{\text{settlement}}] = 30.
\end{cases}
$$

This ensures that actions are made **before** the delivery period, based purely on forecasted expectations.

---

## Expected Profit (Evaluation Phase)

After the decision is made, we evaluate the **expected profit** from each strategy as:

$$
\mathbb{E}[\pi_{\text{buy}}] = p_{\text{pos}}(90 - 30) + (1 - p_{\text{pos}})(0 - 30),
$$
$$
\mathbb{E}[\pi_{\text{sell}}] = p_{\text{pos}}(30 - 90) + (1 - p_{\text{pos}})(30 - 0).
$$

The profit formulas are not used during decision-making — only for **post-hoc analysis** to evaluate the model’s trading performance.



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')
import warnings
# Filter out specific warning message
warnings.filterwarnings("ignore")


loading the data

In [3]:
df = pd.read_csv('./data.csv')
df.rename(columns={'val': 'regulation_mw'}, inplace=True)
df.rename(columns={'utc': 'datetime'}, inplace=True)
df['datetime'] = pd.to_datetime(df['datetime'], utc=True)
df = df.sort_index() # make sure it is in correct time dirction

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np

def prepare_ml_features(df, target_time):
    """
    Create features for predicting whether 17:00-18:00 sum will be positive
    """
    features = []
    targets = []
    
    # For each historical day at 16:53
    for date in df['datetime'].dt.date.unique():  # Exclude today
        
        # Point in time: 16:54 on that date
        cutoff = pd.Timestamp(date).tz_localize('UTC') + pd.Timedelta(hours=16, minutes=53)
        delivery_start = pd.Timestamp(date).tz_localize('UTC') + pd.Timedelta(hours=17)
        delivery_end = pd.Timestamp(date).tz_localize('UTC') + pd.Timedelta(hours=18)
        
        # Only use data available at 16:54
        available_data = df[(df['datetime'].dt.date == date) & (df['datetime'] <= cutoff)]
        
        
        delivery_data = df[(df['datetime'] >= delivery_start) & 
                          (df['datetime'] < delivery_end)]
        if date == pd.Timestamp('2021-02-16').date():
            print(delivery_data)
        # FEATURES (what we know at 16:54)
        feat = {
            # Current state
            'current_reg': available_data.iloc[-1]['regulation_mw'],
            'reg_5min_ago': available_data.iloc[-5]['regulation_mw'] if len(available_data) > 5 else 0,
            'reg_10min_ago': available_data.iloc[-10]['regulation_mw'] if len(available_data) > 10 else 0,
            
            # Recent trends
            'last_5min_mean': available_data.tail(5)['regulation_mw'].mean(),
            'last_10min_mean': available_data.tail(10)['regulation_mw'].mean(),
            'last_30min_mean': available_data.tail(30)['regulation_mw'].mean(),
            'last_60min_mean': available_data.tail(60)['regulation_mw'].mean(),
            
            
            # Momentum
            'momentum_10min': available_data.tail(10)['regulation_mw'].mean() - 
                             available_data.tail(20).head(10)['regulation_mw'].mean(),
            
            # Time features
            'weekday': pd.Timestamp(date).weekday(),
            'is_monday': 1 if pd.Timestamp(date).weekday() == 0 else 0,
            'is_friday': 1 if pd.Timestamp(date).weekday() == 4 else 0,
            
            # Today's pattern
            'morning_avg': df[(df['datetime'].dt.date == date) & 
                             (df['datetime'].dt.hour < 12)]['regulation_mw'].mean(),
            'afternoon_avg': df[(df['datetime'].dt.date == date) & 
                              (df['datetime'].dt.hour >= 12) & 
                              (df['datetime'].dt.hour < 17)]['regulation_mw'].mean(),
            
            # Volatility
            'last_5min_volatility': available_data.tail(5)['regulation_mw'].std(),
            'last_10min_volatility': available_data.tail(10)['regulation_mw'].std(),
            'last_60min_volatility': available_data.tail(60)['regulation_mw'].std(),
            
            # Extreme indicators
            'recent_max': available_data.tail(60)['regulation_mw'].max(),
            'recent_min': available_data.tail(60)['regulation_mw'].min(),
        }
        
        # TARGET: Was the sum positive?
        target = 1 if delivery_data['regulation_mw'].sum() > 0 else 0
        
        features.append(feat)
        targets.append(target)
    
    return pd.DataFrame(features[:-1]), np.array(targets[:-1]), pd.DataFrame([features[-1]])

# Prepare training data
current_time = '2021-02-16 16:54:00' 
X, y, pred_X = prepare_ml_features(df, current_time)

print(f"Training samples: {len(X)}")
print(f"Positive cases: {y.sum()} ({y.mean():.1%})")

Empty DataFrame
Columns: [datetime, regulation_mw]
Index: []
Training samples: 412
Positive cases: 209 (50.7%)


In [5]:
split_idx = int(len(X) * 0.7)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss

results = {}
# 1. Logistic Regression
logreg = LogisticRegression(random_state=42)
logreg.fit(X_train, y_train)
logreg_pred = logreg.predict(X_test)
logreg_proba = logreg.predict_proba(X_test)[:, 1]

results['Logistic Regression'] = {
    'predictions': logreg_pred,
    'accuracy': accuracy_score(y_test, logreg_pred),
    'precision': precision_score(y_test, logreg_pred),
    'recall': recall_score(y_test, logreg_pred),
    'f1': f1_score(y_test, logreg_pred),
    'roc_auc': roc_auc_score(y_test, logreg_proba),
    'log_loss': log_loss(y_test, logreg_proba)
}

xgb = XGBClassifier(
    objective='binary:logistic',
    n_estimators=500,
    learning_rate=0.05,
    max_depth=4,
    random_state=42
)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)
xgb_proba = xgb.predict_proba(X_test)[:, 1]

results['XGBoost Classifier'] = {
    'predictions': xgb_pred,
    'accuracy': accuracy_score(y_test, xgb_pred),
    'precision': precision_score(y_test, xgb_pred),
    'recall': recall_score(y_test, xgb_pred),
    'f1': f1_score(y_test, xgb_pred),
    'roc_auc': roc_auc_score(y_test, xgb_proba),
    'log_loss': log_loss(y_test, xgb_proba)
}

`accuracy`: Overall proportion of correct predictions.

`precision`: Of all predicted positives, how many were actually positive (Good for avoiding false positives.)

`recall`: Of all actual positives, how many did you correctly identify (Good for avoiding false negatives.)

`f1`: Harmonic mean of precision and recall — balances both.

`roc_auc`: Measures how well the model separates classes across all thresholds. Higher is better.

`log_loss`: Penalizes incorrect confident predictions — lower is better.

In [7]:
for model_name, metrics in results.items():
    print(f"\n{model_name}:")
    print(f"  Accuracy:   {metrics['accuracy']:.3f}")
    print(f"  Precision:  {metrics['precision']:.3f}")
    print(f"  Recall:     {metrics['recall']:.3f}")
    print(f"  F1 Score:   {metrics['f1']:.3f}")
    print(f"  ROC AUC:    {metrics['roc_auc']:.3f}")
    print(f"  Log Loss:   {metrics['log_loss']:.4f}")



Logistic Regression:
  Accuracy:   0.669
  Precision:  0.685
  Recall:     0.735
  F1 Score:   0.709
  ROC AUC:    0.660
  Log Loss:   0.6380

XGBoost Classifier:
  Accuracy:   0.565
  Precision:  0.617
  Recall:     0.544
  F1 Score:   0.578
  ROC AUC:    0.561
  Log Loss:   0.9913


Since Logistic Regression performs better in this case, we will use it to train on the entire dataset and make predictions for 2021-02-16.

In [8]:
logreg = LogisticRegression(random_state=42)
logreg.fit(X, y)
logreg_pred = logreg.predict(pred_X)
logreg_proba = logreg.predict_proba(pred_X)[:, 1]

In [236]:
# model = XGBClassifier(
#     objective='binary:logistic',
#     n_estimators=500,
#     learning_rate=0.01,
#     max_depth=3,
#     early_stopping_rounds=100,
#     random_state=42
# )
# model.fit(X, y,
#           eval_set=[(X, y)],
#         verbose=100)

[0]	validation_0-logloss:0.69104
[100]	validation_0-logloss:0.57174
[200]	validation_0-logloss:0.51509
[300]	validation_0-logloss:0.48135
[400]	validation_0-logloss:0.45467
[499]	validation_0-logloss:0.43021


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=100,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, num_parallel_tree=None, ...)

In [9]:
def trading_decision(prob_pos, buy_price=30.0, sell_price=30.0, high_settlement=90.0, low_settlement=0.0):
    

    # Expected settlement price (weighted by probability)
    expected_settlement = prob_pos * high_settlement + (1 - prob_pos) * low_settlement

    # Decision logic
    if expected_settlement > buy_price:
        action = "BUY"
        expected_profit = expected_settlement - buy_price
    elif expected_settlement < sell_price:
        action = "SELL"
        expected_profit = sell_price - expected_settlement
    else:
        action = "HOLD"
        expected_profit = 0.0

    return action, expected_profit, expected_settlement

p_pos = logreg_proba 
action, profit, exp_price = trading_decision(logreg_proba)

print(f"Model probability: {p_pos[0]:.2f}")
print(f"Expected settlement: €{exp_price[0]:.2f}")
print(f"Decision: {action} | Expected profit: €{profit[0]:.2f}/MWh")



Model probability: 0.52
Expected settlement: €46.39
Decision: BUY | Expected profit: €16.39/MWh


Also we can see which features are more important in order

In [10]:
importance = pd.Series(logreg.coef_[0], index=X.columns)

# Sort by absolute impact
importance_sorted = importance.abs().sort_values(ascending=False)
print(importance_sorted)

weekday                  0.029867
last_10min_mean          0.028781
is_monday                0.012666
reg_5min_ago             0.011081
last_60min_mean          0.007881
morning_avg              0.006584
reg_10min_ago            0.006460
is_friday                0.005468
last_30min_mean          0.004865
last_5min_mean           0.004390
last_60min_volatility    0.003797
last_10min_volatility    0.003647
last_5min_volatility     0.002410
recent_max               0.002225
momentum_10min           0.000758
current_reg              0.000728
afternoon_avg            0.000440
recent_min               0.000178
dtype: float64
